We begin by importing the core libraries: 
pandas for data manipulation, numpy for numerical operations, and matplotlib for future visualizations.

Monitoring the library version ensures consistency across different environments.

In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.__version__


'2.3.3'

We load the raw ai_job_dataset.csv into a DataFrame. 

This is the first official step of our ETL process, moving data from a static file into an active environment where we can begin our work.

In [136]:
df=pd.read_csv("ai_job_dataset.csv")

Before diving deep, we use .shape to see the scale of the landscape. 

This tells us exactly how many job postings (rows) and attributes (columns) we are working with.

In [137]:
df.shape

(15000, 19)

A quick "sanity check" using .head() allows us to see the first five rows. 

This confirms the data loaded correctly and gives us a glimpse into the raw formatting of the values.

In [138]:
df.head()

,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


What exactly are we measuring? By listing the columns, we are defining the "characters" of our story.
 
From salaries to locations and skill sets, these headers represent the specific lenses through which we will view the global AI market.

In [139]:
df.columns

Index(['job_id', 'job_title', 'salary_usd', 'salary_currency',
       'experience_level', 'employment_type', 'company_location',
       'company_size', 'employee_residence', 'remote_ratio', 'required_skills',
       'education_required', 'years_experience', 'industry', 'posting_date',
       'application_deadline', 'job_description_length', 'benefits_score',
       'company_name'],
      dtype='object')

Data integrity is key. 

We check for duplicate rows to ensure our analysis isn't skewed by "noise" or repeated postings, making sure every entry represents a unique opportunity.

In [140]:
print("number of duplicate rows", df.duplicated().sum())

number of duplicate rows 0


No dataset is perfect. By summing null values, we locate the gaps in our information so we can decide whether to fill them in or remove them to maintain high data quality.

In [141]:
print(df.isnull().sum())

job_id                    0
job_title                 0
salary_usd                0
salary_currency           0
experience_level          0
employment_type           0
company_location          0
company_size              0
employee_residence        0
remote_ratio              0
required_skills           0
education_required        0
years_experience          0
industry                  0
posting_date              0
application_deadline      0
job_description_length    0
benefits_score            0
company_name              0
dtype: int64


To analyze trends over time, we need to treat dates as a sequence rather than just text.

We use pd.to_datetime with errors='coerce' to convert string dates into datetime64 objects, ensuring that any unparseable dates are safely handled.

In [142]:
df['posting_date'] = pd.to_datetime(df['posting_date'], errors='coerce')
df['application_deadline'] = pd.to_datetime(df['application_deadline'] ,errors='coerce')

Columns like salary_usd and benefirts_score are often imported as generic text. We cast these to numeric types to ensure that mathematical operations—like calculating average salaries or experience requirements—are accurate and computationally efficient.

In [143]:
df['salary_usd'] = pd.to_numeric(df['salary_usd'] , errors='coerce')
df['benefits_score']= pd.to_numeric(df['benefits_score'] , errors='coerce')

Raw datasets often use cryptic abbreviations (like 'SE' or 'FT') for efficiency. We map these codes to human-readable labels such as "Senior-level" or "Full-Time." This transforms "machine-speak" into a clear narrative that is easy for anyone to understand.

In [144]:
experience_map = {
    'EN': 'Entry-level',
    'MI': 'Mid-level',
    'SE': 'Senior-level',
    'EX': 'Executive'
}

df['experience_level'] = df['experience_level'].replace(experience_map)

company_map = {
    'M': 'Medium',
    'L': 'Large',
    'S': 'Small',
    
}

df['company_size'] = df['company_size'].replace(company_map)

employment_type_map = {
    'FT': 'Full-Time',
    'PT': 'Part-Time',
    'CT': 'Contract',
    'FL': 'Freelance'
}

df['employment_type'] = df['employment_type'].replace(employment_type_map)

remote_map = {
    0: 'Onsite',
    50: 'Hybrid',
    100: 'Remote'
}

df['remote_ratio'] = df['remote_ratio'].replace(remote_map)

We perform a "spot check" by viewing unique combinations of our newly labeled categories. 

This confirms that our mapping logic was applied correctly across the dataset and that the new labels look exactly as expected.

In [145]:
df[['experience_level', 'employment_type', 'remote_ratio']].drop_duplicates().head(10)

,experience_level,employment_type,remote_ratio
0,Senior-level,Contract,Hybrid
1,Entry-level,Contract,Remote
2,Mid-level,Freelance,Onsite
3,Senior-level,Freelance,Hybrid
4,Entry-level,Part-Time,Remote
7,Entry-level,Freelance,Onsite
8,Senior-level,Contract,Onsite
9,Senior-level,Part-Time,Onsite
10,Senior-level,Part-Time,Remote
11,Executive,Contract,Onsite


As the final step of this phase, we check the data types of every column. 

This provides technical confirmation that our cleaning was successful: dates should now be datetime64 and salaries should be float64 or int64.

In [146]:
print(df.dtypes)

job_id                            object
job_title                         object
salary_usd                         int64
salary_currency                   object
experience_level                  object
employment_type                   object
company_location                  object
company_size                      object
employee_residence                object
remote_ratio                      object
required_skills                   object
education_required                object
years_experience                   int64
industry                          object
posting_date              datetime64[ns]
application_deadline      datetime64[ns]
job_description_length             int64
benefits_score                   float64
company_name                      object
dtype: object


Extreme values can skew our statistics. We use the Interquartile Range (IQR) method to identify and remove statistical outliers in salary and experience, ensuring our final insights represent the "typical" AI job market.

In [147]:
columns_to_check = ['salary_usd'] 

for col in columns_to_check:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
   
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    print(f"Shape after removing {col} outliers:", df.shape)

Shape after removing salary_usd outliers: (14517, 19)


After removing outliers, we use .describe() to generate a summary of our numerical data. 

This is a vital check to ensure that our means, medians, and ranges (especially for salary) now fall within realistic, expected boundaries for the AI job market.

In [148]:
df.describe()

,salary_usd,years_experience,posting_date,application_deadline,job_description_length,benefits_score
count,14517.000000,14517.000000,14517,14517,14517.000000,14517.000000
mean,109246.604533,5.982228,2024-08-29 10:17:05.170489600,2024-10-11 21:46:23.137011712,1503.655645,7.505208
min,32519.000000,0.000000,2024-01-01 00:00:00,2024-01-16 00:00:00,500.000000,5.000000
25%,69423.000000,1.000000,2024-04-29 00:00:00,2024-06-13 00:00:00,1004.000000,6.300000
50%,97365.000000,4.000000,2024-08-28 00:00:00,2024-10-12 00:00:00,1511.000000,7.500000
75%,140193.000000,9.000000,2024-12-29 00:00:00,2025-02-10 00:00:00,2001.000000,8.800000
max,260729.000000,19.000000,2025-04-30 00:00:00,2025-07-11 00:00:00,2499.000000,10.000000
std,50652.355882,5.404928,NaN,NaN,576.148389,1.448920


We use .info() to get a high-level technical summary. 

This confirms our row count is correct after filtering and verifies that every column is stored in the proper format (integers, floats, or objects), ensuring the dataset is lightweight and efficient for the next phase.

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14517 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   job_id                  14517 non-null  object        
 1   job_title               14517 non-null  object        
 2   salary_usd              14517 non-null  int64         
 3   salary_currency         14517 non-null  object        
 4   experience_level        14517 non-null  object        
 5   employment_type         14517 non-null  object        
 6   company_location        14517 non-null  object        
 7   company_size            14517 non-null  object        
 8   employee_residence      14517 non-null  object        
 9   remote_ratio            14517 non-null  object        
 10  required_skills         14517 non-null  object        
 11  education_required      14517 non-null  object        
 12  years_experience        14517 non-null  int64      

Before we export, we run one last scan for missing values. 
This ensures that our transformation and outlier removal processes didn't accidentally introduce new "gaps" in the data, guaranteeing a 100% clean dataset for our final report.

In [150]:
df.isnull().sum()

job_id                    0
job_title                 0
salary_usd                0
salary_currency           0
experience_level          0
employment_type           0
company_location          0
company_size              0
employee_residence        0
remote_ratio              0
required_skills           0
education_required        0
years_experience          0
industry                  0
posting_date              0
application_deadline      0
job_description_length    0
benefits_score            0
company_name              0
dtype: int64

Our ETL process is complete. We export the cleaned, transformed, and validated dataset to a new CSV file. 

This "Gold Standard" version of the data is now ready for visualization and business intelligence reporting.

In [151]:
df.to_csv("cleaned_dataset.csv", index=False, encoding='utf-8-sig')

Great data science requires great documentation. Here, we programmatically build a Data Dictionary that defines every column, its data type, and a real-world example. 
We export this to Excel so that any stakeholder or teammate can immediately understand the context of our variables without reading the raw code.

In [152]:
column_descriptions = {
    'job_title': 'Job title related to AI and data roles',
    'salary_usd': 'Annual salary expressed in USD',
    'country': 'Country where the job is located',
    'company_location': 'Location of the hiring company',
    'year': 'Year of the job posting',
    'experience_level': 'Level of professional experience (Entry-level, Mid-level, Senior-level, Executive)',
    'employment_type': 'Type of employment contract (Full-Time, Part-Time, Contract, Freelance)',
    'remote_ratio': 'Numeric indicator of remote work (0 = Onsite, 50 = Hybrid, 100 = Remote)',
    'work_mode': 'Descriptive work mode derived from remote_ratio (Onsite, Hybrid, Remote)',
    'job_count': 'Job demand counter used for aggregation in BI analysis'
}

data_dict = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': [df[col].dtype for col in df.columns],
    'Description': [column_descriptions.get(col, 'Additional job-related attribute') for col in df.columns],
    'Example Value': [
        df[col].dropna().iloc[0] if not df[col].dropna().empty else None
        for col in df.columns
    ]
})

data_dict.to_excel("ai_jobs_data_dictionary.xlsx", index=False)